In [1]:
# install the open cv library
!pip install opencv-python

In [2]:
# importing the packages
import pandas as pd
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [3]:
# there are approx 5 features present overall across 75000 images
features_dataframe = pd.read_csv("final_dataset_containing_5_extracted_features.csv")

In [4]:
features_dataframe.shape

(72986, 12)

In [5]:
# viewing the features_dataframe
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
0,00000.png,125,101.0,73.0,NaN,NaN,89,130.0,87.0,NaN,NaN,No
1,000004.png,127,75.0,NaN,NaN,NaN,90,87.0,NaN,NaN,NaN,Yes
2,00001.png,126,75.0,76.0,NaN,NaN,91,90.0,139.0,NaN,NaN,No
3,000014.png,130,77.0,NaN,NaN,NaN,89,91.0,NaN,NaN,NaN,Yes
4,000016.png,127,173.0,74.0,NaN,NaN,90,61.0,91.0,NaN,NaN,Yes


In [6]:
# performing shuffling on features_dataframe
features_dataframe = features_dataframe.sample(frac=1)
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
36384,15458.png,126,75.0,81.0,NaN,NaN,90,90.0,143.0,NaN,NaN,No
58032,38266.png,95,127.0,NaN,NaN,NaN,125,87.0,NaN,NaN,NaN,No
68545,49308.png,122,76.0,NaN,NaN,NaN,88,88.0,NaN,NaN,NaN,No
6339,01927.png,73,128.0,81.0,NaN,NaN,141,89.0,92.0,NaN,NaN,No
15014,045211.png,86,130.0,116.0,NaN,NaN,139,92.0,138.0,NaN,NaN,Yes


In [7]:
features_dataframe = features_dataframe.fillna(-1)

In [8]:
# normalizing the data
is_gan = features_dataframe["is_gan"] 
features_dataframe.drop(columns=["file_name", "is_gan"], inplace=True)

In [9]:
standard_scalar = StandardScaler()
features_dataframe = standard_scalar.fit_transform(features_dataframe)
features_dataframe = pd.DataFrame(features_dataframe)
features_dataframe

,0,1,2,3,4,5,6,7,8,9
0,0.751776,-0.300489,0.704683,-0.432465,-0.189726,-0.188229,0.156418,1.459072,-0.420531,-0.183362
1,-0.506174,1.064378,-0.923653,-0.432465,-0.189726,1.943607,0.070425,-0.899031,-0.420531,-0.183362
2,0.589460,-0.274241,-0.923653,-0.432465,-0.189726,-0.310048,0.099089,-0.899031,-0.420531,-0.183362
3,-1.398913,1.090625,0.704683,-0.432465,-0.189726,2.918160,0.127754,0.623910,-0.420531,-0.183362
4,-0.871386,1.143120,1.399704,-0.432465,-0.189726,2.796341,0.213747,1.377193,-0.420531,-0.183362
...,...,...,...,...,...,...,...,...,...,...
72981,0.832934,-0.274241,0.863545,2.578717,-0.189726,-0.310048,0.099089,1.606453,2.741093,-0.183362
72982,0.711197,-0.195499,1.002549,2.951784,-0.189726,-0.370958,0.099089,1.671956,1.261184,-0.183362
72983,-1.236597,-2.295293,-0.923653,-0.432465,-0.189726,-0.188229,-2.452038,-0.899031,-0.420531,-0.183362
72984,-1.196018,1.011883,1.121696,-0.432465,-0.189726,-0.431868,0.099089,1.409944,-0.420531,-0.183362


In [10]:
features_dataframe["is_gan"] = is_gan
features_dataframe["is_gan"] = features_dataframe["is_gan"].map(dict(Yes=1, No=0))
features_dataframe

,0,1,2,3,4,5,6,7,8,9,is_gan
0,0.751776,-0.300489,0.704683,-0.432465,-0.189726,-0.188229,0.156418,1.459072,-0.420531,-0.183362,0
1,-0.506174,1.064378,-0.923653,-0.432465,-0.189726,1.943607,0.070425,-0.899031,-0.420531,-0.183362,1
2,0.589460,-0.274241,-0.923653,-0.432465,-0.189726,-0.310048,0.099089,-0.899031,-0.420531,-0.183362,0
3,-1.398913,1.090625,0.704683,-0.432465,-0.189726,2.918160,0.127754,0.623910,-0.420531,-0.183362,1
4,-0.871386,1.143120,1.399704,-0.432465,-0.189726,2.796341,0.213747,1.377193,-0.420531,-0.183362,1
...,...,...,...,...,...,...,...,...,...,...,...
72981,0.832934,-0.274241,0.863545,2.578717,-0.189726,-0.310048,0.099089,1.606453,2.741093,-0.183362,0
72982,0.711197,-0.195499,1.002549,2.951784,-0.189726,-0.370958,0.099089,1.671956,1.261184,-0.183362,0
72983,-1.236597,-2.295293,-0.923653,-0.432465,-0.189726,-0.188229,-2.452038,-0.899031,-0.420531,-0.183362,0
72984,-1.196018,1.011883,1.121696,-0.432465,-0.189726,-0.431868,0.099089,1.409944,-0.420531,-0.183362,0


In [11]:
features_dataframe.isna().sum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
is_gan    0
dtype: int64

In [12]:
x = features_dataframe
y = features_dataframe["is_gan"]
x.drop(columns=["is_gan"], inplace=True)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

In [14]:
f"Shape of training data = {x_train.shape[0]} and testing data = {x_test.shape[0]}"

'Shape of training data = 51090 and testing data = 21896'

## Performing experiments using the following hyperparameters to find the best model

In [15]:
# Performing Kfold cross validation
rf_pipeline = Pipeline([("classifier", RandomForestClassifier())])
param_grid = {'classifier__n_estimators' : [50, 100, 150, 200],
             'classifier__criterion' : ["gini", "entropy", "log_loss"],
             'classifier__max_depth' : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
             'classifier__max_features': ["auto", "sqrt", "log2"],
             'classifier__n_jobs': [-1]}

rf_pipeline

Pipeline(steps=[('classifier', RandomForestClassifier())])

In [16]:
gridsearch_cv = GridSearchCV(rf_pipeline, param_grid=param_grid, cv=10, verbose=2)

In [17]:
gridsearch_cv.fit(x_train, y_train)

Fitting 10 folds for each of 684 candidates, totalling 6840 fits
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=auto, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   3.7s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=auto, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.5s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=auto, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.5s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=auto, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.5s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=auto, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.5s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=auto, classifier__n_estimato

[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=sqrt, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   0.9s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=sqrt, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   0.9s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=sqrt, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   1.0s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=sqrt, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   1.1s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=sqrt, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   1.1s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=sqrt, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   0.9s
[CV] END c

[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=log2, classifier__n_estimators=150, classifier__n_jobs=-1; total time=   1.5s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=log2, classifier__n_estimators=150, classifier__n_jobs=-1; total time=   1.3s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=log2, classifier__n_estimators=150, classifier__n_jobs=-1; total time=   1.2s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=log2, classifier__n_estimators=150, classifier__n_jobs=-1; total time=   1.1s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=log2, classifier__n_estimators=150, classifier__n_jobs=-1; total time=   1.1s
[CV] END classifier__criterion=gini, classifier__max_depth=2, classifier__max_features=log2, classifier__n_estimators=150, classifier__n_jobs=-1; total time=   1.6s
[CV] END c

[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.2s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.7s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.1s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   1.7s
[CV] END c

[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.8s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.6s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.8s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.8s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.6s
[CV] END classifier__criterion=gini, classifier__max_depth=3, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   0.7s
[CV] END classif

...

[CV] END classifier__criterion=log_loss, classifier__max_depth=19, classifier__max_features=log2, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.1s
[CV] END classifier__criterion=log_loss, classifier__max_depth=19, classifier__max_features=log2, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=19, classifier__max_features=log2, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=19, classifier__max_features=log2, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.2s
[CV] END classifier__criterion=log_loss, classifier__max_depth=19, classifier__max_features=log2, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.1s
[CV] END classifier__criterion=log_loss, classifier__max_depth=19, classifier__max_features=log2, classifier__n_estimators=200, classifier__n_jobs=-1;

[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=auto, classifier__n_estimators=200, classifier__n_jobs=-1; total time=   2.1s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=sqrt, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=sqrt, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=sqrt, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=sqrt, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=sqrt, classifier__n_estimators=50, classifier__n_jobs=-1; tota

[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=log2, classifier__n_estimators=50, classifier__n_jobs=-1; total time=   1.9s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=log2, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   1.9s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=log2, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=log2, classifier__n_estimators=100, classifier__n_jobs=-1; total time=   2.0s
[CV] END classifier__criterion=log_loss, classifier__max_depth=20, classifier__max_features=log2, classifier__n_estimators=100, classifier__n_jobs=-1; t

/Users/shrirang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2280 fits failed out of a total of 6840.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2280 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/shrirang/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 436, in _process_worker
    r = call_item()
  File "/Users/shrirang/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 288, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/shrirang/opt/anaconda3/lib/python3.9/site-pac

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                   11, 12, 13, 14, 15, 16, 17,
                                                   18, 19, 20],
                         'classifier__max_features': ['auto', 'sqrt', 'log2'],
                         'classifier__n_estimators': [50, 100, 150, 200],
                         'classifier__n_jobs': [-1]},
             verbose=2)

In [18]:
gridsearch_cv.best_params_

{'classifier__criterion': 'gini',
 'classifier__max_depth': 10,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 150,
 'classifier__n_jobs': -1}

## Training the model on the above best parameters

In [25]:
rfclf = RandomForestClassifier(criterion="gini", max_depth=10, n_jobs=-1, max_features="log2", n_estimators=150, oob_score=True, random_state=42)
rfclf.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=150,
                       n_jobs=-1, oob_score=True, random_state=42)

In [26]:
rf_predictions = rfclf.predict(x_test)

In [27]:
cm = metrics.confusion_matrix(y_test, rf_predictions)
print(f"The confusion matrix for RF Clf is given below: \n {cm}")

The confusion matrix for RF Clf is given below: 
 [[14908     0]
 [ 6988     0]]


In [28]:
rf_precision = precision_score(y_test, rf_predictions, average='weighted', labels=np.unique(rf_predictions))
rf_precision

0.6808549506759225

In [29]:
rf_recall = recall_score(y_test, rf_predictions, average='weighted', labels=np.unique(rf_predictions))
rf_recall

1.0

In [30]:
rf_f1 = f1_score(y_test, rf_predictions, average='weighted', labels=np.unique(rf_predictions))
rf_f1

0.8101293337680686

In [31]:
# storing the svm model
filename = 'random_forest_final.pkl'
pickle.dump(logisticRegr, open(filename, 'wb'))

NameError: name 'logisticRegr' is not defined